In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot

In [86]:
kerdo_iv = pd.read_csv("kerdo_iv.csv")

In [38]:
kerdo_iv.head(5)

,Időbélyeg,1. Mennyire rendszeresen látogatod az előadásokat?,2. Készítesz jegyzetet az előadásokon?,3. Bejársz jelenléti nélküli gyakorlatokra?,4. Miből szoktál tanulni? (leginkább),"5. Egyedül vagy csoportosan, másokkal együtt tanulsz inkább?",6. Tanulás során készítesz saját jegyzetet?,7. Ha van ajánlott irodalom a tárgyhoz akkor bele szoktál olvasni?,8. Egy évközbeni zhra mennyivel hamarabb kezdesz el készülni?,9. Vizsgára mennyivel hamarabb kezdesz el tanulni?,10. A nap folyamán mikor tudsz a legeredményesebben tanulni?,"11. Számonkérés előtt pár órával, az utolsó pillanatokban szoktál e még tanulni?",12. Hallgatsz zenét tanultás közben?,13. Hol laksz?,"14. Reprodukálod önállóan egy gyakorlati feladat megoldását, a megoldókulcsot nem nézve?",Hány kreditet szereztél előző félévben?
0,2015/12/05 6:49:21 du. CET,Néha,Nem,Rendszeresen,Diasorok,Csoportosan,Néha,Nem,Pár nappal,1 héttel,Délután,Rendszeresen,Néha,Albérlet,Néha,26 - 30 kredit
1,2015/12/05 6:49:21 du. CET,Néha,Igen,Rendszeresen,Mások által készített jegyzetből,Egyedül,Néha,Igen,Pár nappal,1 héttel,Éjszaka / este,Rendszeresen,Rendszeresen,Kollégiumban,Rendszeresen,0 - 10 kredit
2,2015/12/05 6:49:57 du. CET,Néha,Nem,Rendszeresen,Diasorok,Egyedül,Néha,Nem,1 héttel,1 héttel,Délután,Rendszeresen,Néha,Albérlet,Soha,26 - 30 kredit
3,2015/12/05 6:50:03 du. CET,Néha,Nem,Néha,VikWikis anyag,Csoportosan,Néha,Nem,Pár nappal,1 héttel,Éjszaka / este,Rendszeresen,Rendszeresen,Kollégiumban,Néha,31 - 37 kredit
4,2015/12/05 6:50:35 du. CET,Néha,Nem,Néha,VikWikis anyag,Egyedül,Néha,Nem,1 héttel,1 héttel,Éjszaka / este,Rendszeresen,Rendszeresen,Otthon,Néha,26 - 30 kredit


In [95]:
features = kerdo_iv

# soha/néha/rendszeresen értékek átírása 0/1/2 -re

In [96]:
soha_indexes = features == "Soha"

In [97]:
features[soha_indexes] = 0

In [98]:
neha_indexes = features == "Néha"

In [99]:
features[neha_indexes] = 1

In [100]:
rendszeresen_indexes = features == "Rendszeresen"

In [101]:
features[rendszeresen_indexes] = 2

# igen/nem átírása 1/0 értékre

In [103]:
nem_indexes = features == "Nem"

In [104]:
igen_indexes = features == "Igen"

In [105]:
features[nem_indexes] = 0

In [106]:
features[igen_indexes] = 1